In [ ]:
import os
import sys
from tqdm.notebook import tqdm
import numpy as np
from scipy.signal import lti, lsim, welch
if not '..' in sys.path:
    sys.path.append('..')
from pfcommon import OU_2, combine_output_spectra

dB = 10

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import FixedLocator, NullLocator, FixedFormatter
import seaborn as sns

fontsize = 9
lw = 0.75
matplotlib.rc('font', **{'family': 'Times', 'size': fontsize})
matplotlib.rc('axes', **{'linewidth': 0.75, 'labelsize': fontsize})
matplotlib.rc('xtick', **{'labelsize': fontsize})
matplotlib.rc('ytick', **{'labelsize': fontsize})
matplotlib.rc('xtick.major', **{'width': lw, 'size':3})
matplotlib.rc('ytick.major', **{'width': lw, 'size':3})
matplotlib.rc('ytick.minor', **{'width': lw, 'size':1.5})

Load the data:

In [ ]:
N = 100
data_file = os.path.join('..','data','Sardinia','SM_configs_from_data','001',
                         'V2020_Rete_Sardegna_2021_06_03cr_AC_TF_-6.0_2.0_{}.npz'.format(N))
data = np.load(data_file, allow_pickle=True)
F = data['F']
TF = data['TF'][1:,:,:]
OUT = data['OUT'][1:,:,:]
var_names = list(data['var_names'])
load_names = list(data['load_names'])[1:]
N_loads = len(load_names)
# power flow solution
PF = data['PF'].item()
# power absorbed by the loads
P_loads = np.array([PF['loads'][name]['P'] for name in load_names]) # [MW]
# figure labels
labels = ['{}: {:.1f} MW'.format(name.split('__')[0][4:9], P) for name,P in zip(load_names, P_loads)]

In [ ]:
var_type = 'm:ur'
if var_type == 's:xspeed':
    var_name = 'Grid-CODCTI-CODCTI0201________SUBNET__-CODCTI0201GGR1____GEN_____.ElmSym.speed'
elif var_type == 'm:fe':
#     var_name = 'Grid-BNFC_I-BNFC_I0601________SUBNET__-BNFC_I0601A1______BUS_____.ElmTerm.fe'
    var_name = 'Grid-CODCTI-CODCTI3801________SUBNET__-CODCTI3801B1______BUS_____.ElmTerm.fe'
elif var_type == 'm:ur':
#     var_name = 'Grid-BNFC_I-BNFC_I0601________SUBNET__-BNFC_I0601A1______BUS_____.ElmTerm.ur'
    var_name = 'Grid-CODCTI-CODCTI3801________SUBNET__-CODCTI3801B1______BUS_____.ElmTerm.ur'
else:
    raise Exception(f"Unknown variable type '{var_type}'")
IDX = var_names.index(var_name)

In [ ]:
device_name = os.path.splitext(var_name)[0]
F0 = 50.
OUT_multi = combine_output_spectra(OUT,
                                   var_type,
                                   [device_name],
                                   var_names,
                                   data['ref_SM_name'].item(),
                                   data['F'],
                                   F0,
                                   data['PF'].item(),
                                   data['bus_equiv_terms'].item(),
                                   dB=None)
out_multi = OUT_multi[device_name]

In [ ]:
fig,ax = plt.subplots(1, 1, figsize=(5,3.5))
cmap = plt.get_cmap('viridis', N_loads)
ax.semilogx(F, dB*np.log10(np.abs(out_multi)), 'k', lw=2, label='Total')
for i in range(N_loads):
    ax.semilogx(F, dB*np.log10(np.abs(OUT[i,:,IDX])), color=cmap(i), lw=1, label=labels[i])
ax.set_xlabel('Frequency [Hz]')
ax.legend(loc='lower left', frameon=False, fontsize=9)
ax.set_ylabel('PSD [dB]')
sns.despine()
fig.tight_layout()

Performs the vector fitting for a given number of poles:

In [ ]:
def run_vf(X, F, n_poles, n_iter=3, weights=None, poles_guess=None, do_plot=False):
    Y = X.astype(np.complex128)
    if weights is None:
        weights = np.ones(F.size, dtype=np.float64)
    else:
        assert weights.size == F.size
        weights = weights.astype(np.float64)

    F0,F1 = np.log10(F[[0,-1]])
    s = (2j*np.pi*F).astype(np.complex128)

    import vectfit3 as vf
    opts = vf.opts.copy()
    opts['asymp'] = 2
    opts['skip_res'] = True  # skip residue computation
    opts['spy2'] = False     # do not plot the results

    # initial guess for pole positions
    if poles_guess is not None:
        poles = poles_guess
    else:
        # logarithmically evenly spaced poles in the range [F0,F1]
        poles = -2*np.pi*np.logspace(F0, F1, n_poles, dtype=np.complex128)
    for i in range(n_iter):
        if i == n_iter-1:
            opts['skip_res'] = False
            opts['spy2'] = do_plot
        SER,poles,rmserr,fit = vf.vectfit(Y, s, poles, weights, opts)
    return SER,poles,rmserr,fit

Choose the number of poles based on a threshold of the RMS error:

In [ ]:
N_poles = np.zeros(N_loads, dtype=int)
SER = []
poles = []
rms_err = np.zeros(N_loads)
rms_thresh = np.zeros(N_loads)
fit = np.zeros((N_loads, F.size), dtype=complex)
systems = []
max_N_poles = 50
for i in range(N_loads):
    rms_thresh[i] = 10 ** (np.floor(np.log10(np.abs(TF[i,:,IDX]).mean())) - 3)
    for n in range(max_N_poles):
        ser,p,rms_err[i],fit[i,:] = run_vf(TF[i,:,IDX], F, n+1)
        if abs(rms_err[i]) < rms_thresh[i]:
            break
    SER.append(ser)
    poles.append(p)
    N_poles[i] = n+1
    systems.append(lti(SER[i]['A'],SER[i]['B'],SER[i]['C'],SER[i]['D']))
    print('[{:2d}] # of poles sufficient to have an RMS error below {:g}: {}.'.\
          format(i+1, rms_thresh[i], N_poles[i]))

The results of the vector fitting:

In [ ]:
fig,ax = plt.subplots(1, 1, figsize=(5,3.5))
for i in range(N_loads):
    col = np.array(cmap(i)) + 0.3
    col[col > 1] = 1
    ax.plot(F, dB*np.log10(np.abs(TF[i,:,IDX])), color=cmap(i), lw=2, label=labels[i])
    ax.plot(F, dB*np.log10(np.abs(fit[i])), '--', color=col, lw=1)
ax.set_xscale('log')
ax.set_xlabel('Frequency [Hz]')
ax.legend(loc='lower left', frameon=False, fontsize=9)
ax.set_ylabel('PSD [dB]')
sns.despine()
fig.tight_layout()

Generate an OU process with the appropriate statistics:

In [ ]:
tend = 10800
srate = 200
dt = 1/srate
mean,stddev,tau = 0,0.1*P_loads,20e-3
μ,c,α = mean,stddev*np.sqrt(2/tau),1/tau
cutoff = α/(2*np.pi)
time = np.r_[0 : tend+dt/2 : dt]
N_samples = time.size
U = np.zeros((N_loads,N_samples))
for i in tqdm(range(N_loads)):
    U[i,:] = OU_2(dt, α, μ, c[i], N_samples)

Filter the OU process with the TFs extracted above:

In [ ]:
Y = []
for i in tqdm(range(N_loads)):
    _,y,_ = lsim(systems[i], U[i,:], time)
    assert y.imag.max() < 1e-10
    Y.append(y.real)
Y = np.array(Y)

Plot the OU process and the corresponding filtered signal:

In [ ]:
ds = 100
fig,ax = plt.subplots(N_loads, 2, figsize=(8,1*N_loads), sharex=True)
for i in range(N_loads):
    ax[i,0].plot(time[::ds], U[i,::ds], 'k', lw=0.5, label='U')
    ax[i,1].plot(time[::ds], Y[i,::ds], 'r', lw=0.5, label='Y')
    ax[i,0].set_ylabel('U [MW]')
    ax[i,1].set_ylabel('Y [MW]')
for a in ax[-1,:]:
    a.set_xlabel('Time [s]')
sns.despine()
fig.tight_layout()

Compute the PSDs:

In [ ]:
window = 200 / dt
onesided = True
def run_welch(x, dt, window, onesided):
    freq,P = welch(x, 1/dt, window='hamming',
                   nperseg=window, noverlap=window/2,
                   return_onesided=onesided, scaling='density')
    if onesided:
        P /= 2
    else:
        Nf = freq.size
        freq = freq[:Nf//2]
        P = P[:Nf//2]
    return freq, P, np.sqrt(P)

P_U,abs_U = [],[]
for i in range(N_loads):
    _,pu,absu = run_welch(U[i,:], dt, window, onesided)
    P_U.append(pu)
    abs_U.append(absu)
P_U,abs_U = np.array(P_U),np.array(abs_U)
P_U_dB = dB*np.log10(P_U)
abs_U_dB = dB*np.log10(abs_U)

P_Y,abs_Y = [],[]
for i in range(N_loads):
    _,py,absy = run_welch(Y[i,:], dt, window, onesided)
    P_Y.append(py)
    abs_Y.append(absy)
P_Y_dB = dB*np.log10(P_Y)
abs_Y_dB = dB*np.log10(abs_Y)
freq,P_Y_multi,abs_Y_multi = run_welch(Y.sum(axis=0), dt, window, onesided)
P_Y_multi_dB = dB*np.log10(P_Y_multi)
abs_Y_multi_dB = dB*np.log10(abs_Y_multi)

P_U_theor = np.array([(ci/α)**2 / (1 + (2*np.pi*F/α)**2) for ci in c])
P_U_theor_dB = dB*np.log10(P_U_theor)
abs_U_theor = np.sqrt(P_U_theor)
abs_U_theor_dB = dB*np.log10(abs_U_theor)

abs_TFxU = np.abs(TF[:,:,IDX]) * abs_U_theor
P_TFxU = abs_TFxU**2
abs_TFxU_dB = dB*np.log10(abs_TFxU)
P_TFxU_dB = dB*np.log10(P_TFxU)

Plot the PSDs:

In [ ]:
fig,ax = plt.subplots(1, 2, figsize=(8,4), sharex=True)

for i in range(N_loads):
    col = np.array(cmap(i)[:3]) + 0.3
    col[col>1] = 1
    ax[0].plot(freq, abs_U[i,:], color=cmap(i), lw=0.75, label=labels[i])
    ax[0].plot(F, abs_U_theor[i,:], color=cmap(i), lw=2)
    ax[1].plot(F, np.abs(OUT[i,:,IDX]), color=cmap(i), lw=1)

ax[0].plot(cutoff+np.zeros(2), ax[0].get_ylim(), '--', color=[1,0,0], lw=1, label='OU cutoff')
ax[1].plot(F, np.abs(out_multi), 'k', lw=2, label='OUT')
ax[1].plot(freq, abs_Y_multi, color=[1,.5,0], lw=1, label='Y', alpha=0.75)

for a in ax:
    a.legend(loc='upper left', frameon=False)
    a.set_xscale('log')
    a.set_xlabel('Frequency [Hz]')
ax[0].set_ylabel('|U(j$\omega$)|')

sns.despine()
fig.tight_layout()

In [ ]:
i = 0
gray = 0.7 + np.zeros(3)
fig,ax = plt.subplots(3, 1, figsize=(3.5,4.5), sharex=True)
ax[0].plot(freq[1:], P_U_dB[i,1:], color=gray, lw=0.75, label='OU')
ax[0].plot(F, P_U_theor_dB[i,:], 'k', lw=2, label='OU theor.')
ax[0].plot(cutoff+np.zeros(2), ax[0].get_ylim(), '--', color='k',
           lw=1, label='OU cutoff')
ax[1].plot(F, dB*np.log10(np.abs(TF[i,:,IDX])), 'k', lw=2, label='TF')
ax[1].plot(F, dB*np.log10(np.abs(fit[i,:])), color=gray,
           lw=1, label=f'Fit (# poles = {N_poles[i]})')
ax[2].plot(freq, P_Y_dB[i,:], color=gray, lw=1, label='OUT')
ax[2].plot(F, P_TFxU_dB[i,:], 'k', lw=1, label='TF x OU theor.')

ax[0].set_ylabel('PSD')
ax[1].set_ylabel(r'|Y(j$\omega$)| [dB{}]'.format(dB))
ax[2].set_ylabel('PSD')
ticks = np.logspace(-3,2,6)
for a in ax:
    a.xaxis.set_major_locator(FixedLocator(ticks))
    a.xaxis.set_minor_locator(NullLocator())
    a.set_xscale('log')
    a.grid(which='major', axis='x', ls=':', lw=0.5, color=[.6,.6,.6])
ax[0].legend(loc='lower left', frameon=False, fontsize=8)
ax[1].legend(loc='best', bbox_to_anchor=(0.55, 0.6, 0.5, 0.5), frameon=False, fontsize=8)
ax[2].legend(loc='lower left', frameon=False, fontsize=8)
ax[-1].set_xlim(ticks[[0,-1]])
ax[-1].set_xlabel('Frequency [Hz]')
sns.despine()
fig.tight_layout()
plt.savefig('spectra_input_output_{}.pdf'.format(labels[i].replace(':','').replace(' ','_')))